In [2]:
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, MaxPooling1D, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, CSVLogger
import keras_metrics

from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np
import pickle

In [30]:
#from cleaner import Cleaner
#cleaner = Cleaner()
#cleaner.create_tokenizer_and_clean()

from cleaner_n_grams import Cleaner_ngrams
cleaner = Cleaner_ngrams()
cleaner.create_tokenizer_and_clean()

cleaning the text data for vectorizer
loading CountVectorizer


In [3]:
filename = "../data/json_bundle_reviews/large-bundle-clean.json"
df = pd.read_json(filename)
#df = self.clean_news(df)

y = df.sentiment.values
sentences = df['content'].values
sentences.shape


(100267,)

In [16]:
print("loading vectorizer, train data and test data")
sentences_train, sentences_test, y_train, y_test = train_test_split(
sentences, y, test_size=0.25, random_state=1000)
maxlen = 250
with open('../data/neural_network_config/ngram_vectorizer.pickle', 'rb') as handle:
        ngram_vectorizer = pickle.load(handle)

loading vectorizer, train data and test data
loaded data


In [4]:
print("separating")
X_train =ngram_vectorizer.transform(sentences_train)
X_test =ngram_vectorizer.transform(sentences_test)
print(X_train.toarray())
X_train =pad_sequences(X_train, value=0., maxlen=maxlen)
X_test =pad_sequences(X_test, value=0., maxlen=maxlen)
print("loaded data")

separating


NameError: name 'ngram_vectorizer' is not defined

In [16]:
filter_length = 300

        #model = Sequential()
#model.add(Embedding(vocab_size, 20, input_length=maxlen))
#model.add(Dropout(0.15))
#model.add(GlobalMaxPool1D())
#model.add(Dense(output_size, activation='sigmoid'))
vocab_size = len(ngram_vectorizer.vocabulary_) + 1  # Adding 1 because of reserved 0 index

model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=maxlen))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(Conv1D(filter_length, 5, activation='relu'))
#model.add(MaxPooling1D(5))
#model.add(Conv1D(filter_length, 5, activation='relu'))
#model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])

In [ ]:
print("training model")
csv_logger = CSVLogger('log_loss.csv', append=False, separator=';')

callbacks = [
ModelCheckpoint(filepath='../data/neural_network_config/temp-model.h5', save_best_only=True), csv_logger]

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=40,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks)


training model
Epoch 1/10
1891/1891 [==============================] - 1040s 550ms/step - loss: 0.6932 - accuracy: 0.5040 - precision: 0.5006 - recall: 0.4355 - val_loss: 0.6933 - val_accuracy: 0.4957 - val_precision: 0.4915 - val_recall: 1.0000

In [ ]:
print("saving model")
# serialize model to JSON
model_json = model.to_json()
with open("../data/neural_network_config/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../data/neural_network_config/model.h5")
print("Saved model to disk")